In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import *
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.manifold import TSNE
import umap.umap_ as umap

random_state = 1000

In [ ]:
df = pd.read_csv('../data/rock_data.csv')
df.columns

In [ ]:
# handle missing values
if df.isnull().values.any():
    df.fillna(df.mean(), inplace=True)
    print('Missing values filled with mean.')

In [ ]:
n_clusters = 3
c_map = cm.rainbow(np.linspace(0, 1, n_clusters))
sns.pairplot(df.iloc[:,2:], hue='Class', hue_order=[12, 58, 34], palette={12: c_map[0], 58: c_map[1], 34: c_map[2]}, plot_kws={'alpha':0.2})

In [ ]:
# drop highly correlated features
df.drop(['Area', 'Perimeter','Median Height', 'Mean Height', 'STD Height'], axis=1, inplace=True)
df.columns

In [ ]:
# normalize features
# df.iloc[:,3:] = df.iloc[:,3:].apply(zscore)

In [ ]:
# detect and remove outliers

In [ ]:
n_clusters = 3
X = df.iloc[:,3:].to_numpy(dtype=float)
df_clustered = df.iloc[:,3:].copy()

def map_class_values(x):
    mapping = {12: 0, 58: 1, 34: 2}
    return mapping.get(x,None)

actual_class = np.vectorize(map_class_values)(df['Class'].values)
df_clustered['cluster'] = actual_class
kmeans_sil = silhouette_score(X, actual_class)
samps_sil = silhouette_samples(X, actual_class)

print(f"For ground-truth clustering, the average silhouette_score = {kmeans_sil}")

fig, axs = subplots(nrows=1, ncols=2, figsize=(15,6))

c_map = cm.rainbow(np.linspace(0, 1, n_clusters))

y_lower = 10
for i in range(n_clusters):
    cluster_sil = np.sort(samps_sil[actual_class == i])
    y_upper = y_lower + cluster_sil.shape[0]

    axs[0].fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_sil, color=c_map[i])

    axs[0].set_xlim([-0.2, 1])
    axs[0].set_ylim([0, len(X) + (n_clusters + 1) * 10])
    axs[0].text(-0.05, y_lower + 0.5 * cluster_sil.shape[0], str(i))

    y_lower = y_upper + 10

axs[0].set_xlabel("Silhouette Score")
axs[0].set_ylabel("Cluster Label")

axs[0].axvline(x=kmeans_sil, color="red", linestyle=(3,(5,7)))

axs[0].set_yticks([])
axs[0].set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])

pd.plotting.parallel_coordinates(df_clustered, 'cluster', ax=axs[1], color=c_map, alpha=0.1)
xticks(rotation=90)
axs[1].legend().set_visible(False)

show()

In [ ]:
pca = PCA(n_components=2)
components = pca.fit_transform(X)
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig, axs = subplots(nrows=1, ncols=1, figsize=(6,6))

c_map = cm.rainbow(np.linspace(0, 1, n_clusters))
axs.scatter(components[:,0], components[:,1], color=df_clustered['cluster'].map(lambda x: c_map[x]))
axs.set_xlabel(labels['0'], fontsize=24)
axs.set_ylabel(labels['1'], fontsize=24)

show()

In [ ]:
sort_idx = np.argsort(-np.abs(pca.components_[0]))
for si in sort_idx:
    print(f'{df.columns[3+si]}: {pca.components_[0][si]}')

In [ ]:
sort_idx = np.argsort(-np.abs(pca.components_[1]))
for si in sort_idx:
    print(f'{df.columns[3+si]}: {pca.components_[1][si]}')

In [ ]:
tsne = TSNE(n_components=2)
embedding = tsne.fit_transform(X)
labels = {
    str(i): f"TSNE {i+1}" for i in range(2)
}

fig, axs = subplots(nrows=1, ncols=1, figsize=(6,6))

c_map = cm.rainbow(np.linspace(0, 1, n_clusters))
axs.scatter(embedding[:,0], embedding[:,1], color=df_clustered['cluster'].map(lambda x: c_map[x]))
axs.set_xlabel(labels['0'], fontsize=24)
axs.set_ylabel(labels['1'], fontsize=24)

show()

In [ ]:
fig, axs = subplots(nrows=1, ncols=1, figsize=(6,6))
um = umap.UMAP()
embedding = um.fit_transform(X)
scatter(embedding[:, 0], embedding[:, 1], c=df_clustered['cluster'])